# Experiment 01: Detecting Algorithmic Bias in a Hiring Dataset

---

In [1]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# -- model --
from sklearn.linear_model import LogisticRegression

In [2]:
!pip install fairlearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 4.7 MB/s eta 0:00:0000:01


In [3]:
from fairlearn.metrics import MetricFrame, selection_rate

In [4]:
url = '/kaggle/input/mobile-price-classification/train.csv'
data = pd.read_csv(url)

In [5]:
data.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [6]:
data.columns

Index(['battery_power', 'blue', 'clock_speed', 'dual_sim', 'fc', 'four_g',
       'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height',
       'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g',
       'touch_screen', 'wifi', 'price_range'],
      dtype='object')

In [7]:
features = ["dual_sim", "fc", "four_g", "int_memory", "n_cores", "three_g"]
df = data[features]

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   dual_sim    2000 non-null   int64
 1   fc          2000 non-null   int64
 2   four_g      2000 non-null   int64
 3   int_memory  2000 non-null   int64
 4   n_cores     2000 non-null   int64
 5   three_g     2000 non-null   int64
dtypes: int64(6)
memory usage: 93.9 KB


In [9]:
df = pd.get_dummies(df, drop_first=True)
df.head()

,dual_sim,fc,four_g,int_memory,n_cores,three_g
0,0,1,0,7,2,0
1,1,0,1,53,3,1
2,1,2,1,41,5,1
3,0,0,0,10,6,1
4,0,13,1,44,2,1


In [10]:
dual_sim_col = []
four_g_col = []
three_g_col = []

for v in df["dual_sim"]:
    dual_sim_col.append(str(v))

df["dual_sim"] = dual_sim_col

for v in df["four_g"]:
    four_g_col.append(str(v))

df["four_g"] = four_g_col

for v in df["three_g"]:
    three_g_col.append(str(v))

df["three_g"] = three_g_col

In [11]:
df = pd.get_dummies(df, drop_first=True)
df.head()

,fc,int_memory,n_cores,dual_sim_1,four_g_1,three_g_1
0,1,7,2,False,False,False
1,0,53,3,True,True,True
2,2,41,5,True,True,True
3,0,10,6,False,False,True
4,13,44,2,False,True,True


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   fc          2000 non-null   int64
 1   int_memory  2000 non-null   int64
 2   n_cores     2000 non-null   int64
 3   dual_sim_1  2000 non-null   bool 
 4   four_g_1    2000 non-null   bool 
 5   three_g_1   2000 non-null   bool 
dtypes: bool(3), int64(3)
memory usage: 52.9 KB


In [13]:
X = df.drop(["three_g_1"], axis=1)
y = df["three_g_1"]

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [15]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [16]:
y_pred = model.predict(X_test)
y_pred

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False, False,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True, False,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False,  True, False,  True,  True, False,  True,
        True,  True,  True,  True,  True,  True, False, False,  True,
       False,  True,  True, False,  True,  True,  True,  True, False,
       False,  True,  True, False, False, False,  True,  True,  True,
        True,  True,

In [17]:
# Fairness evaluation
sim = X_test['dual_sim_1']
metric_frame = MetricFrame(metrics=selection_rate,
y_true=y_test,
y_pred=y_pred,
sensitive_features=sim)
print("Selection Rates by Dual Sim:\n", metric_frame.by_group)

Selection Rates by Dual Sim:
 dual_sim_1
False    0.968153
True     0.695804
Name: selection_rate, dtype: float64


#### Conclusion
The selection rate represents the proportion of individuals for whom the model predicted a positive outcome (in this case, having a three-g capability). Looking at the output, it seems there is a significant difference in the selection rate between devices with dual SIM capabilities (True) and those without (False). Devices without dual SIM have a much higher selection rate (0.968) compared to devices with dual SIM (0.696). This suggests a potential bias in the model's predictions regarding the presence of three-g based on whether a device has dual SIM.